# ARMAモデルを全店舗に適用して予測

1. データの前処理（集計期間が店舗ごとに異なるため）
   1. 集計が始まる前のVisitorsをすべてNullにする
   2. 集計期間の中で欠損した日付がある場合は0で補填
2. 店舗ごとにARMAモデルを適用
   1. 差分をとる
   2. ARMAで学習
   3. 学習結果で予測
   4. 差分をもとに戻す
3. Submit file を作成し、出力

In [1]:
import os
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.tsa.api as smt
import statsmodels.api as sm
from statsmodels.tsa.arima_model import AR, ARMA, ARIMA
from sklearn import metrics
from IPython.display import  display
sns.set_style('whitegrid')
%matplotlib inline

In [2]:
data_dir = '../data/'

## データの読み込み
もとデータに存在しなかった日付をvisitors=nanとして行を追加したテーブル

features_format.csv と test_data.csv を縦に結合する

In [3]:
# データ読み込み関数
def import_csv(file_name='features_format', data_dir = '../data/', datetime_keys='visit_date', sort_keys=['air_store_id', 'visit_date']):
    df = pd.read_csv(os.path.join(data_dir, file_name + '.csv'), engine='python')
    df[datetime_keys] = pd.to_datetime(df[datetime_keys])
    df = df.sort_values(by=sort_keys)
    return df
    

In [4]:
features_format = import_csv('features_format')
features_format.head()

,air_store_id,visit_date,visitors
0,air_00a91d42b08b08d9,2016-01-01,NaN
1,air_00a91d42b08b08d9,2016-01-02,NaN
2,air_00a91d42b08b08d9,2016-01-03,NaN
3,air_00a91d42b08b08d9,2016-01-04,NaN
4,air_00a91d42b08b08d9,2016-01-05,NaN


In [5]:
test_data = import_csv('test_data')
test_data.head()

,air_store_id,visit_date,id,visitors
0,air_00a91d42b08b08d9,2017-03-15,air_00a91d42b08b08d9_2017-03-15,42.0
1,air_00a91d42b08b08d9,2017-03-16,air_00a91d42b08b08d9_2017-03-16,39.0
2,air_00a91d42b08b08d9,2017-03-17,air_00a91d42b08b08d9_2017-03-17,37.0
3,air_00a91d42b08b08d9,2017-03-18,air_00a91d42b08b08d9_2017-03-18,4.0
4,air_00a91d42b08b08d9,2017-03-19,air_00a91d42b08b08d9_2017-03-19,NaN


In [6]:
train = pd.concat([features_format, test_data[['air_store_id', 'visit_date', 'visitors']]], axis=0)
train = train.sort_values(by=['air_store_id', 'visit_date'])
train = train.reset_index()
train.head()

,index,air_store_id,visit_date,visitors
0,0,air_00a91d42b08b08d9,2016-01-01,NaN
1,1,air_00a91d42b08b08d9,2016-01-02,NaN
2,2,air_00a91d42b08b08d9,2016-01-03,NaN
3,3,air_00a91d42b08b08d9,2016-01-04,NaN
4,4,air_00a91d42b08b08d9,2016-01-05,NaN


## データの前処理（集計期間が店舗ごとに異なるため）

 1. 集計が始まる前のVisitorsをすべてNullにする
 2. 集計期間の中で欠損した日付がある場合は0で補填

In [7]:
air_store_id = 'air_00a91d42b08b08d9'
# air_store_idで抽出
train_by_store = train[train['air_store_id']==air_store_id]

In [8]:
def fill_closed_date_by_0(train_by_store=train_by_store):
    train_by_store = train_by_store.reset_index(drop=True)
    
    # visitorsの1つ前のレコードを持つ列を追加
    train_by_store['visitors_shift_1'] = train_by_store.shift(1).loc[:,'visitors']
    # visitorsの1つ前のレコードがともにNaNなら0の列を追加
    train_by_store['is_not_both_nan'] = ~(train_by_store['visitors'].isnull() & train_by_store['visitors_shift_1'].isnull())
    # is_not_both_nanの累積和の列を追加（この列はnanが終わるまで0の値になる）
    train_by_store['cumsum_not_nan'] = train_by_store['is_not_both_nan'].cumsum()
    # cumsum_not_nan > 0 ならvisitorsのnanに0を代入
    train_by_store.loc[train_by_store['cumsum_not_nan']>0, 'visitors'] = train_by_store.loc[train_by_store['cumsum_not_nan']>0, 'visitors'].fillna(0)
    # 不要な列を削除
    train_by_store = train_by_store[['air_store_id', 'visit_date', 'visitors']]
    
    return train_by_store

In [9]:
train_by_store = fill_closed_date_by_0(train_by_store)
train_by_store.head()

,air_store_id,visit_date,visitors
0,air_00a91d42b08b08d9,2016-01-01,NaN
1,air_00a91d42b08b08d9,2016-01-02,NaN
2,air_00a91d42b08b08d9,2016-01-03,NaN
3,air_00a91d42b08b08d9,2016-01-04,NaN
4,air_00a91d42b08b08d9,2016-01-05,NaN


In [10]:
train_by_store.dropna().head()

,air_store_id,visit_date,visitors
182,air_00a91d42b08b08d9,2016-07-01,35.0
183,air_00a91d42b08b08d9,2016-07-02,9.0
184,air_00a91d42b08b08d9,2016-07-03,0.0
185,air_00a91d42b08b08d9,2016-07-04,20.0
186,air_00a91d42b08b08d9,2016-07-05,25.0


### 集計の空白期間が長すぎる店舗はないか？

## ARMAモデルを適用

   1. 差分をとる
   2. ARMAで学習
   3. 学習結果で予測
   4. 差分をもとに戻す

In [11]:
air_store_id = 'air_00a91d42b08b08d9'
# air_store_idで抽出
train_by_store = train[train['air_store_id']==air_store_id]

train_by_store = fill_closed_date_by_0(train_by_store)

order = (6,2)

In [12]:
def arma_pred(train_by_store=train_by_store, order=(6,2)):

    # 差分をとる
    visitors_diff = train_by_store['visitors'].diff()
    visitors_diff.name = 'visitors' + '_diff'

    # 差分の列をtrainに結合
    train_by_store = pd.concat([train_by_store, visitors_diff], axis=1)

    # nullのある行を削除
    train_by_store = train_by_store.dropna(axis=0)

    # 学習
    result = ARMA(train_by_store['visitors_diff'].values, order=order, dates=train_by_store['visit_date'], freq='D').fit()

    # 予測
    predicts_diff = result.predict(start='2017-04-23', end='2017-05-31')

    # 差分をもとにもどす
    last_train_value = train_by_store['visitors'].iloc[-1]
    predicts = [predicts_diff[0] + last_train_value]
    for i in range(1, len(predicts_diff)):
        predicts.append(predicts_diff[i] + predicts[-1])
        
    return np.array(predicts)

In [13]:
predict = arma_pred(train_by_store,order=(6,2))
predict[:5]

array([11.74732841, 24.17240857, 29.9765318 , 24.39947992, 35.30016972])

In [14]:
# air_store_idで抽出
#train_by_store = train[train['air_store_id']==air_store_id]

# 差分をとる
visitors_diff = train_by_store['visitors'].diff()
visitors_diff.name = 'visitors' + '_diff'

# 差分の列をtrainに結合
train_by_store = pd.concat([train_by_store, visitors_diff], axis=1)

# nullのある行を削除
train_by_store = train_by_store.dropna(axis=0)

# 学習
result = ARMA(train_by_store['visitors_diff'].values, order=order, dates=train_by_store['visit_date']).fit()

# 予測
predicts_diff = result.predict(start='2017-04-23', end='2017-05-31')

# 差分をもとにもどす
last_train_value = train_by_store['visitors'].iloc[-1]
predicts = [predicts_diff[0] + last_train_value]
for i in range(1, len(predicts_diff)):
    predicts.append(predicts_diff[i] + predicts[-1])

C:\Users\hiroshi_kuriyama\AppData\Local\conda\conda\envs\recruit_py36\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)


In [15]:
predicts

[11.747328407986789,
 24.172408572604944,
 29.976531798188624,
 24.399479920525035,
 35.30016971653667,
 42.72351032661973,
 23.529054289035003,
 16.783402378262124,
 25.105459936558066,
 28.821661663897373,
 27.34148719277564,
 33.37732220654389,
 36.38063609222029,
 25.790749437424456,
 20.780705285833974,
 25.725314774474366,
 28.583043389227722,
 28.594345264117326,
 31.96607233940488,
 33.01075520572084,
 26.874076286630785,
 23.530383665761562,
 26.38113792018616,
 28.587434264712325,
 29.113251860369132,
 30.983945145133063,
 31.186441404668322,
 27.50305372136235,
 25.361492311356553,
 27.018091632765444,
 28.67199137411443,
 29.307012715627824,
 30.33558150447904,
 30.20196871954816,
 27.936064084160638,
 26.590384249748503,
 27.579804810940217,
 28.78872994935382,
 29.370060299921455]

In [16]:
[predicts_diff[0] + last_train_value]

[11.747328407986789]

## 店舗ごとに分析

In [17]:
np.array([datetime.date(2017, 4, 23) + datetime.timedelta(i) for i in range(39)])

array([datetime.date(2017, 4, 23), datetime.date(2017, 4, 24),
       datetime.date(2017, 4, 25), datetime.date(2017, 4, 26),
       datetime.date(2017, 4, 27), datetime.date(2017, 4, 28),
       datetime.date(2017, 4, 29), datetime.date(2017, 4, 30),
       datetime.date(2017, 5, 1), datetime.date(2017, 5, 2),
       datetime.date(2017, 5, 3), datetime.date(2017, 5, 4),
       datetime.date(2017, 5, 5), datetime.date(2017, 5, 6),
       datetime.date(2017, 5, 7), datetime.date(2017, 5, 8),
       datetime.date(2017, 5, 9), datetime.date(2017, 5, 10),
       datetime.date(2017, 5, 11), datetime.date(2017, 5, 12),
       datetime.date(2017, 5, 13), datetime.date(2017, 5, 14),
       datetime.date(2017, 5, 15), datetime.date(2017, 5, 16),
       datetime.date(2017, 5, 17), datetime.date(2017, 5, 18),
       datetime.date(2017, 5, 19), datetime.date(2017, 5, 20),
       datetime.date(2017, 5, 21), datetime.date(2017, 5, 22),
       datetime.date(2017, 5, 23), datetime.date(2017, 5, 24),
 

In [18]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
# air_store_id一覧を用意する
air_store_ids = train['air_store_id'].unique()

predicts_appended = []
air_sotre_id_appended = []
date_appended = []
error_id = []
num_i = 0


for air_store_id in air_store_ids:
    num_i += 1
    air_store_id_i = air_store_id
    
    train_by_store = train[train['air_store_id']==air_store_id]
    train_by_store = fill_closed_date_by_0(train_by_store)
    try:
        predicts = arma_pred(train_by_store, order=(6,2))
    # 例外が発生したら、平均値をpredictsにする
    except:
        error_id.append(air_store_id)
        predicts = np.zeros(39) + train_by_store['visitors'].mean()
        
    predicts_appended = np.append(predicts_appended, predicts)
    air_sotre_id_appended = np.append(air_sotre_id_appended, np.array([air_store_id for i in range(39)]))
    date_appended = np.append(date_appended, np.array([datetime.date(2017, 4, 23) + datetime.timedelta(i) for i in range(39)]))
    

In [20]:
num_i

821

In [21]:
air_store_id_i

'air_fff68b929994bfbd'

In [22]:
fill_closed_date_by_0(train[train['air_store_id']==air_store_id_i]).dropna().head()

,air_store_id,visit_date,visitors
182,air_fff68b929994bfbd,2016-07-01,3.0
183,air_fff68b929994bfbd,2016-07-02,3.0
184,air_fff68b929994bfbd,2016-07-03,0.0
185,air_fff68b929994bfbd,2016-07-04,0.0
186,air_fff68b929994bfbd,2016-07-05,7.0


## 提出ファイルを作成

submissionデータをインポート

In [23]:
sample_submission = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'), engine='python')
sample_submission.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


### 予測の出力をDataFrameにまとめる

In [63]:
predicts_submission_np = np.vstack((air_sotre_id_appended, date_appended, predicts_appended)).T
predicts_submission_df = pd.DataFrame(predicts_submission_np)
predicts_submission_df.columns = ['air_sotre_id', 'date','visitors']
predicts_submission_df['id'] = predicts_submission_df.apply(lambda x : x['air_sotre_id'] + '_' + x['date'].strftime('%Y-%m-%d'), axis=1)
predicts_submission_df.head()

,air_sotre_id,date,visitors,id
0,air_00a91d42b08b08d9,2017-04-23,11.7473,air_00a91d42b08b08d9_2017-04-23
1,air_00a91d42b08b08d9,2017-04-24,24.1724,air_00a91d42b08b08d9_2017-04-24
2,air_00a91d42b08b08d9,2017-04-25,29.9765,air_00a91d42b08b08d9_2017-04-25
3,air_00a91d42b08b08d9,2017-04-26,24.3995,air_00a91d42b08b08d9_2017-04-26
4,air_00a91d42b08b08d9,2017-04-27,35.3002,air_00a91d42b08b08d9_2017-04-27


### 形式がsample_submissionと一致しているか確かめる(結果が0ならOK)

In [62]:
(sample_submission['id'] != predicts_submission_df['id']).sum()

0

予測結果が0以下の場合は0に変換

In [66]:
predicts_submission_df.loc[predicts_submission_df['visitors']<0, 'visitors'] = 0

In [68]:
predicts_submisson = predicts_submission_df[['id', 'visitors']]

In [70]:
predicts_submisson.to_csv(os.path.join(data_dir, 'predicts_submisson_arma.csv'), index=False)